In [40]:
import numpy as np
import scipy.integrate as integrate
from numpy.linalg import inv
import math as m
from functions import *
import importlib

%load_ext autoreload

%autoreload 2

np.set_printoptions(suppress=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This next block will create a class for all of our inputs to be placed into. This will allow for all inputs to have easily accessed names, values, and units.

In [41]:
class Input:
    def __init__(self, name, value, unit):
        self.name = name
        self.value = value
        #self.valueType = type(value)
        self.unit = unit
    
    def printValues(self):
        print("\n")
        for attr, value in self.__dict__.items():
            print(f"{attr}: {value}")

# Please fill in all inputs and their units below
# v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v v 

Units Set: ksi, kip, in, in2

In [42]:
YoungsModulus = 30000
YM_Unit = "ksi"

Area = 1
A_Unit = "in2"

#is it symmetric? if so, about what element?
Symmetric = True
SymmetryAbout = 11

P = -20
Force_Unit = "kip"

# node connections for each element
nArray = np.array([[1,2], [1,3], [2,3], [2,4], [3,4], [3,5], [4,5], [4,6], [5,6], [5,7], [6,7]])

# list of lengths corresponding to each element
# the length of element n is contained in lengths
lArray = np.array([5, 3, 4.8023431781, 3.25, 3.75, 3, 3.905124838, 3.25, 2.5, 3, 3.25, 3.25, 1.25, 3, 3.25])
lArray *= 1000

Length_unit = "mm"

# list of angles relative to positive x axis for each element
theta = m.asin(5/13)
aArray = np.array([(1.5*m.pi), 0, -(m.atan(3.75/3)), theta, 1.5*m.pi, 0, -(m.atan(2.5/3)), theta, 1.5*m.pi, 0, -(m.atan(1.25/3)), theta, 1.5*m.pi, 0, theta])

Angle_unit = "rad"

# input boundary conditions below
# for displacements: [node#, u, v]
# for forces: [node#, Fx, Fy]

uv_BCs = np.array([[1, 0, 0],[2, 0, None]])

force_BCs = np.array([[1, None, None], [2, None, 0], [3, 0, P],[5, 0, P], [7, 0, P], [9, 0, P]])

if len(aArray) != len(lArray) or len(nArray) != len(lArray) or len(nArray) != len(aArray):
    print("Error! Inputs do not have the same amount of nodes or elements!")

Error! Inputs do not have the same amount of nodes or elements!


# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [43]:
E = Input("Young's Modulus", YoungsModulus, YM_Unit)
E.printValues()

if Symmetric == False:
    A = Input("Area", Area, A_Unit)
    A.printValues()

nodes = Input("Node numbers", nArray, "None")
nodes.printValues()

lengths = Input("Lengths", lArray, Length_unit)
lengths.printValues()

angles = Input("Element Angles (relative to the positive x axis)", aArray, Angle_unit)
angles.printValues()



name: Node numbers
value: [[1 2]
 [1 3]
 [2 3]
 [2 4]
 [3 4]
 [3 5]
 [4 5]
 [4 6]
 [5 6]
 [5 7]
 [6 7]]
unit: None


name: Lengths
value: [5000.        3000.        4802.3431781 3250.        3750.
 3000.        3905.124838  3250.        2500.        3000.
 3250.        3250.        1250.        3000.        3250.       ]
unit: mm


name: Element Angles (relative to the positive x axis)
value: [ 4.71238898  0.         -0.89605538  0.39479112  4.71238898  0.
 -0.69473828  0.39479112  4.71238898  0.         -0.39479112  0.39479112
  4.71238898  0.          0.39479112]
unit: rad


number of elements is the number of rows in nodes
<br>number of nodes is the maximum number found in nodes

In [44]:
numElems = len(nodes.value)
print(f"Number of elements: {numElems}")
numNodes = nodes.value.max()
print(f"Number of nodes: {numNodes}")

Number of elements: 11
Number of nodes: 7


Symmetry Code

In [45]:
if Symmetric == True:
    areas = np.full((numElems), Area, dtype=float)
    areas[SymmetryAbout - 1] = Area/2

    print(areas)

    A = Input("Area", areas, A_Unit)
    A.printValues()

[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5]


name: Area
value: [1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5]
unit: in2


two reactions per node
<br>uvList -> list of unknowns (node displacements)
<br>we can input our known values -> fixed_nodes at nodes 4 & 5
<br>these fixed nodes numbers are taken straight from diagram (not adjusted for python counting)
<br><br>u and v for any given node is given as (node# - 1)* 2 and ((node# -1)* 2) + 1 respectively.


In [46]:
uvList = [None]*(numNodes*2)
print("displacement bcs: [node number, x disp (u), y disp (v)]")
print(uv_BCs)

adjust_array(uvList, uv_BCs, "displacements")
print("u & v list:")
print(uvList)

displacement bcs: [node number, x disp (u), y disp (v)]
[[1 0 0]
 [2 0 None]]
adjusting array to account for: displacements boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
done adjusting displacements array!
u & v list:
[0, 0, 0, None, None, None, None, None, None, None, None, None, None, None]


Same procedure as displacement BCs for forces
<br>Input BCs given as : [node #, x force, y force]

In [47]:
#forceList = [0]*(numNodes*2)
forceList = np.zeros(((numNodes*2), 1))

nodesWithLoads = []

loadedNodes = 0

for i in range(len(force_BCs)):
    if (force_BCs[i,1]!= None and force_BCs[i,1] != 0) or (force_BCs[i,2] != None and force_BCs[i,2] != 0):
        loadedNodes += 1
        nodesWithLoads.append((force_BCs[i,0] - 1))


print(f"number of nodes with concentrated loads: {loadedNodes}")
print(f"at nodes: {nodesWithLoads} (Python adjusted nodes)")

#adjust force array given concentrated loading conditions
adjust_array(forceList, force_BCs, "forces")

print(f"force list: \n{forceList}")

number of nodes with concentrated loads: 4
at nodes: [2, 4, 6, 8] (Python adjusted nodes)
adjusting array to account for: forces boundary conditions
case 1 for BC 1
Case 2 for BC 1
case 1 for BC 2
case 4 for BC 2
case 3 for BC 3
Case 2 for BC 3
case 3 for BC 4
Case 2 for BC 4
case 3 for BC 5
Case 2 for BC 5


IndexError: index 16 is out of bounds for axis 0 with size 14

Global [k] matrix is a symmetric array of shape (numNodes x 2, numNodes x 2)
<br>Bar Constants are one per element, stored in a numElemes * 1 array.

In [ ]:
globalK = np.zeros(((numNodes*2),(numNodes*2)))
print(f"Global K array dimensions: {globalK.shape}")

barConstant = np.zeros((numElems,1))
for i in range(numElems):
    barConstant[i] = (E.value*A.value)/lengths.value[i]

print(f"Bar Constants = \n{barConstant}")

## Creating the finite element equation for each Truss Element
### Combining into the global K array

In [ ]:
for i in range(numElems):
    kMatrix = create_truss_k(angles.value[i])
    kMatrix *= barConstant[i]
    print(f"\nk matrix for element {i+1}: \n{kMatrix}")
    #print(kMatrix)
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starting at node: {startNode} and ending at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    #first, adjust 0,0 -> then, 2,0; 0,2; and 2,2
    block_array_adjust(Sn, Sn, globalK, 0, 0, kMatrix)
    block_array_adjust(En, Sn, globalK, 2, 0, kMatrix)
    block_array_adjust(Sn, En, globalK, 0, 2, kMatrix)
    block_array_adjust(En, En, globalK, 2, 2, kMatrix)

print(f"\nGlobal K matrix: \n{globalK}")
unmodified_globalK = globalK.copy()
check_Symmetric(globalK)

In [ ]:
j = 0
while j < numNodes*2:
    print(f"{j}")
    #if j in nodesWithLoads:
     #   j+=2
    #first case: displacement BC = 0. In this case, set combined force values to 0, and alter 
    #globalK to reflect the changes.
    if uvList[j] == 0:
        print(f"u = 0 at row: {j}")
        forceList[j] = 0 
        globalK[j] = 0
        globalK[j,j] = 1
        j+=1
    elif uvList[j] == None:
        print(f"none detected at row {j}")
        j+=1
    else: #when we have a known, non-zero displacement boundary condition
        #alter global stiffness to all zeros and one 1, and set force equal to displacement BC
        print(f"row {j} altered")
        print(uvList[j])
        forceList[j] = uvList[j]
        for c in range(numNodes):
            if (globalK[j,c] > 0) == True:
                globalK[j,c] = 1
            else:
                globalK[j,c] = 0
        j+=1

                
print(globalK)
print(forceList)

In [ ]:
#globalK.round(0)
globalK_inv = inv(globalK)
#print(globalK_inv)

node = 1

uFinal = np.matmul(globalK_inv, forceList)

for i in range(len(uFinal)):
    if check_even(i) == True:
        print(f"u{node} = {uFinal[i]}")
    else:
        print(f"v{node} = {uFinal[i]}")
        node += 1 

Forces can be found from [K]*[u]

In [ ]:
forcesFinal = np.matmul(unmodified_globalK, uFinal)
for i in range(len(forceList)):
    if forceList[i] != forcesFinal[i] and forceList[i] != 0 and forceList[i] != None:
        forcesFinal[i] = forceList[i]
print(f"Final Forces: \n{forcesFinal}")

node = 1

for i in range(len(forcesFinal)):
    if check_even(i) == True:
        print(f"fx{node} = {forcesFinal[i]}")
    else:
        print(f"fy{node} = {forcesFinal[i]}")
        node += 1 


## Finding Stresses

In [ ]:
stressFinal = np.zeros((numElems, 1))
print(stressFinal)
for i in range(numElems):
    startNode = nodes.value[i,0]
    endNode = nodes.value[i,1]
    print(f"Element {i+1} starts at node: {startNode} and ends at node: {endNode}")
    #print(np.vsplit(kMatrix, 2))
    Sn = (startNode - 1)*2
    En = (endNode - 1)*2
    print(f"adjusted start node: {Sn} adjusted end node: {En}")
    trigMatrix = create_truss_stress_trig_matrix(angles.value[i])
    print(f"trigMatrix = \n{trigMatrix}")
    local_uMatrix = np.array([
        uFinal[Sn], 
        uFinal[(Sn + 1)], 
        uFinal[En], 
        uFinal[(En+1)]
        ])
    print(f"local u matrix: \n{local_uMatrix}")
    localLength = np.array([[(-1/lengths.value[i]), 1/lengths.value[i]]])
    print(f"local length matrix: \n{localLength}")

    indStress = (trigMatrix @ local_uMatrix)
    indStress = localLength @ indStress
    indStress *= E.value

    stressFinal[i] = indStress
    
    
    print(f"Stress in element = {indStress}")

    

In [ ]:
print(stressFinal)

In [ ]:
np.set_printoptions(suppress=True, formatter={'float':"{:0.4f}".format})
printTrussNodalDisplacements(uFinal, lengths.unit)
printStresses(stressFinal,E.unit)